In [1]:
# New method for detecting earthquakes in the seismic waveform (extracting a charactristic function for earthquakes detection)
# Outputs are: 1- Text file of the charactristic function (detected transient signals) 2- Mseed file of the charactristic function


import glob
import obspy
import librosa
import numpy as np
from obspy import read
import librosa.display
from obspy import UTCDateTime, read, Trace, Stream
import datetime

# Go through the directory
directory= 'Detect and time earthquakes.ipynb
Transient signal detection.ipynb
folder= glob.glob (directory)

for file in folder:
    
    # Convert julian date
    date= datetime.datetime.strptime(file[-14:-9], '%y%j').date()
    time=date.strftime('%Y-%m-%d')
    
    ### Separation with high precision in time for detecting transient signals
    # Choose a small n_fft for high precision in time domain

    k=1
    sr=16000
    hop_length=32
    win_length=128
    n_fft= 128
    y, sr = librosa.load( file, sr=sr)

    # Compute the spectrogram magnitude and phase
    S_full, phase = librosa.magphase(librosa.stft(y, n_fft= n_fft, hop_length=hop_length, win_length=win_length))
    
    # We'll compare frames using cosine similarity, and aggregate similar frames by taking their (per-frequency) median value.
    S_filter = librosa.decompose.nn_filter(S_full,aggregate=np.median,metric='cosine',width=int(librosa.time_to_frames(k, sr=sr)))
    
    # The output of the filter shouldn't be greater than the input
    S_filter = np.minimum(S_full, S_filter)
    margin_i, margin_v = 2, 10
    power = 2

    # Once we have the masks, simply multiply them with the input spectrogram to separate the components
    mask_i = librosa.util.softmask(S_filter,margin_i * (S_full - S_filter),power=power)
    mask_v = librosa.util.softmask(S_full - S_filter,margin_v * S_filter,power=power)

    S_foreground = mask_v * S_full
    S_background = mask_i * S_full

    # Decompose spectrograms into harmonic and percussive components
    H2, P2 = librosa.decompose.hpss(S_foreground)
    
    # save P2 (percussive component of transient spectrogram) as a charactristic function for detecting earthquakes (as text file and mseed file). 
    with open ( time + '\P2.txt','w') as h:
        out = P2[5:32,:].sum(axis=0)  # only sum P2 in a frequancy range of [3.8,24.6], and set other frequency ranges to zero
        h.write("\n".join(map(lambda x: '%f' % x, out)))
    
    file1 = obspy.read (file[:-9]+'-resp.mseed')
    stats = file1[0].stats
    stats.sampling_rate = 3.125
    stats.npts = len(out)
    stats.location= 'Eq'
    st = Stream([Trace(data=out, header=stats)])
    st.write( time + '\P2.mseed', format='MSEED', encoding=4, reclen=4096)